<a href="https://colab.research.google.com/github/Allen-John-Manoj/Fashion-MNIST-CNN/blob/main/MNIST_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [ ]:
#TF_ENABLE_ONEDNN_OPTS=1
#pip install intel-tensorflow

The above can only be run in Intel openAPI Jupyter notebook. Recommended for faster and better results

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

import numpy as num
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.utils.multiclass import unique_labels

In [ ]:
(train, trainlabel), (test, testlabel) = tf.keras.datasets.fashion_mnist.load_data()

train = train.astype('float32') / 255.0
test = test.astype('float32') / 255.0

train = num.expand_dims(train, -1)
test = num.expand_dims(test, -1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
cnn = tf.keras.models.Sequential()

cnn.add(Conv2D(
           filters=64,
            kernel_size=(5, 5),
           padding='same',
            activation='relu',
            kernel_regularizer=l2(0.01),
           input_shape=(28, 28, 1)))

cnn.add(MaxPooling2D( pool_size=(2, 2), strides=(2, 2)))

cnn.add(BatchNormalization(momentum=0.9))

cnn.add(Conv2D(
           filters=128,
            kernel_size=(3, 3),
            padding='same',
            activation='relu',
            input_shape=(28, 28, 1)))

cnn.add(MaxPooling2D( pool_size=(2, 2)))

#cnn.add(BatchNormalization(momentum=0.9))

cnn.add(Conv2D(
           filters=256,
            kernel_size=(2, 2),
            padding='same',
            activation='relu',
            input_shape=(28, 28, 1)))

cnn.add(MaxPooling2D( pool_size=(2, 2)))


cnn.add(Dropout( rate=0.25 ))

cnn.add(Flatten())

cnn.add(Dense(
        units=256, activation='relu'))

cnn.add(Dense(
       units=100, activation='relu'))

cnn.add(Dense(
        units=10, activation='softmax'))

In [ ]:
cnn.compile(optimizer=Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

cnn.summary()

In [ ]:
history = cnn.fit(
            train, trainlabel,
            #steps_per_epoch = 200,
            batch_size=356,
            epochs=10,
            validation_split=0.2)

test_loss, test_accuracy = cnn.evaluate(test, testlabel, verbose=0)

print(f'\nTest loss: {test_loss:.4f}')
print(f'Test accuracy: {test_accuracy:.4f}')

In [ ]:
predictions = cnn.predict(test)
print(f"Accuracy: {num.max(predictions)*100}")
pred = num.argmax(predictions, axis=1)
cm = confusion_matrix(testlabel, pred)
classes = unique_labels(testlabel, pred)

plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Greens)
plt.title('Confusion Matrix')
plt.colorbar()

report = classification_report(testlabel, pred)
print(report)

tick_marks = num.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)

thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.tight_layout()
plt.show()

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
cnn.save('MNIST_model.h5')

In [ ]:
from google.colab import files

files.download("MNIST_model.h5")

In [ ]:
model = tf.keras.models.load_model('MNIST_model.h5')
tf.saved_model.save(model, 'Conversion.pb')

In [ ]:
!python -m venv openvino_env

In [ ]:
pip install openvino-dev==2023.0.1

In [ ]:
from openvino.tools.mo import convert_model
ov_model = convert_model('Conversion.pb')